In [49]:
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
deployment_name = os.getenv('DEPLOYMENT_NAME')

# A Few Shot Learning

In [50]:
# Zero-shot classification
system_prompt ="""Predict up to 5 emojis as a response to a text chat message. The output
should only include emojis.

input: The new visual design is blowing my mind 🤯
output: ➕,💘, ❤‍🔥

input: Well that looks great regardless
output: ❤️,🪄

input: Unfortunately this won't work
output: 💔,😔

input: sounds good, I'll look into that
output: 🙏,👍

input: 10hr cut of jeff goldblum laughing URL
output: 😂,💀,⚰️
"""
user_prompt = "The new user interface is amazing!"
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}])
print(response.choices[0].message.content)

👏,🎉,🌟,😍,👍


# Chain of Thought

Experiment results demonstrate Zero-shot-CoT using single prompt template, significantly outperform zero-shot LLM performance on diverse benchmark reasoning tasks.  Without any hand-crafted few-shot examples, e.g. increasing the accuracy on MultiArith from 17.7% to 78.7% and GSM8K from 10.4% to 40.7% with large InstructGPT model (text-davinci-002).

Source: [Large Language Models are Zero-Shot Reasoners](https://arxiv.org/abs/2205.11916)

In [51]:
def call_openai(prompt, temperature=1, max_token=60):
    response = client.chat.completions.create(
        model=deployment_name,
        messages = prompt,
            max_tokens=max_token,
            temperature = temperature
    )
    return response.choices[0].message.content

In [52]:
# This prompt gets wrong answer - ZERO_SHOT Prompt

system_prompt = """Answer the following questions:"""
quiz = """The bakers at the Beverly Hills Bakery baked 200 loaves of bread on Monday morning.
They sold 93 loaves in the morning and 39 loaves in the afternoon.
A grocery store returned 6 unsold loaves.
How many loaves of bread did they have left?"""

prompt = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": quiz}
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)

To find out how many loaves of bread the Beverly Hills Bakery had left, we need to subtract the loaves sold and the unsold loaves from the total number of loaves baked.

Total loaves baked: 200
Loaves sold in the morning: 93
Loaves sold in the afternoon: 39
Unsold loaves returned: 6

Total loaves sold: 93 + 39 = 132
Total loaves returned: 6

Total loaves left: 200 - 132 - 6 = 62

Therefore, the Beverly Hills Bakery had 62 loaves of bread left.


In [53]:
# Still wrong answer with few-shot learning

prompt=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Q: Roger has 5 tennis balss. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does Roger have now?"},
    {"role": "assistant", "content": "A: The answer is 11."},
    {"role": "user", "content": quiz}
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)

To find out how many loaves of bread the bakery had left, we need to subtract the loaves sold and the loaves returned from the total loaves baked.

Total loaves baked: 200
Loaves sold in the morning: 93
Loaves sold in the afternoon: 39
Loaves returned: 6

Total loaves left = 200 - 93 - 39 - 6
Total loaves left = 62

Therefore, the Beverly Hills Bakery had 62 loaves of bread left.


In [60]:
# With CoT, the answer is correct

cot_message = " Understand the sentences well and let’s think step by step."

prompt=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": quiz + cot_message}
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)

1. The bakers baked 200 loaves of bread on Monday morning.
2. They sold 93 loaves in the morning and 39 loaves in the afternoon.
3. So, the total loaves sold during the day = 93 + 39 = 132 loaves.
4. The number of loaves left after selling = 200 - 132 = 68 loaves.
5. A grocery store returned 6 unsold loaves.
6. Therefore, the final number of loaves left = 68 + 6 = 74 loaves.

So, the Beverly Hills Bakery had 74 loaves of bread left at the end of the day.


In [55]:
# PROMPT_FEW_SHOT_CoT

prompt=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": """Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"""},
    {"role": "assistant", "content": """Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11."""},
    {"role": "user", "content": quiz + cot_message}
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)

Step 1: The bakers baked 200 loaves of bread.
Step 2: They sold 93 loaves in the morning and 39 loaves in the afternoon.
Step 3: They had 200 - (93 + 39) = 200 - 132 = 68 loaves left.
Step 4: The grocery store returned 6 unsold loaves.
Step 5: They had 68 + 6 = 74 loaves left.
Therefore, they had 74 loaves of bread left.


# Commonsense Reasoning

Paper: [Generated Knowledge Prompting for Commonsense Reasoning](https://arxiv.org/abs/2110.08387)



Provide knowledge, turn knowledge question into reasoning. In general, more knowledge, better result.

3 Contributing factors:

(i) the quality of knowledge, 

(ii) the quantity of knowledge where the performance improves with more knowledge statements, and 

(iii) the strategy for integrating knowledge during inference

In [56]:
common_question = """The player with the lowest score wins.
Is this true or false: Part of golf is trying to get a higher point total than others.
"""

prompt = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=100)

print(response_msg)

False. In golf, the player with the lowest score wins. The objective is to complete the course with the fewest number of strokes possible.


In [57]:
common_question = """A tripod is a kind of easel
How many legs does an easel have?
"""
prompt = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=100)

print(response_msg)

A tripod is actually a type of camera stand with three legs. An easel, on the other hand, is a stand used for displaying or supporting artwork. Typically, an easel has four legs.


## Check out follow 2 examples

In [58]:
# High confidence answer
common_question = """Question: Part of golf is trying to get a higher point total than others. Yes or No?
Knowledge: The objective of golf is to play a set of holes in the least number of strokes. A round of golf typically consists of 18 holes. Each hole is played once in the round on a standard golf course. Each stroke is counted as one point, and the total number of strokes is used to determine the winner of the game.
Explain and Answer: 
"""
prompt=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=100)

print(response_msg)

No, the statement is incorrect. In golf, the objective is to play a set of holes in the least number of strokes possible. The goal is to have the lowest score, not a higher point total than others. Each stroke is counted as one point, and the player with the fewest total strokes at the end of the round is the winner.


In [59]:
# Low confidence answer. 예문을 더 애매하게 바꿔야... 잘못된 결과가 나올것 같은데... 정상으로 나옴.
common_question = """Question: Part of golf is trying to get a higher point total than others. Yes or No?
Knowledge: Golf is a precision club-and-ball sport in which competing players (or golfers) use several types of clubs to hit a ball into a series of holes on a course using the fewest number of strokes. A goal is highest rated with the lowest score calculated by adding up the total number of strokes taken on each hole. The player with the lowest score wins the game.
Explain and Answer: 
"""
prompt=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=100)

print(response_msg)

No, that statement is incorrect. In golf, the goal is to have the lowest score possible by completing each hole in the fewest number of strokes. The player with the lowest total score at the end of the game is the winner. So, in golf, the objective is to have a lower score than others, not a higher point total.
